In [171]:
import pandas as pd
import pathlib

In [172]:
rawdata=pd.read_excel(pathlib.Path.cwd().parents[0].joinpath('data','Tn_library_DB_qval_log2FC.xlsx'))
binary=pd.read_excel(pathlib.Path.cwd().parents[0].joinpath('data','Tn_library_DB.xlsx'))

In [173]:
binary.head()

,Rv_ID,gene_name,2003A_Sassetti,2003B_Sassetti,2005_Rengarajan,2006_Joshi_GI_1,2006_Joshi_GI_2,2011_Griffin_2,2012_Zhang,2013_DeJesus,...,Rv0950,Rv3717,Rv3594,Rv3916c,marP,Rv0954,Rv1432,Rv1096,Rv2680,Rv0307c
0,Rv0001,dnaA,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Rv0002,dnaN,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Rv0003,recF,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,Rv0004,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rv0005,gyrB,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [174]:
rawdata.head()

,Rv_ID,gene_name,2003A_Sassetti_q_val,2003A_Sassetti_log2FC,2003B_Sassetti_q_val,2003B_Sassetti_log2FC,2005_Rengarajan_q_val,2005_Rengarajan_log2FC,2006_Joshi_GI_1_q_val,2006_Joshi_GI_1_log2FC,...,Rv1096_q_val,Rv1096_log2FC,Rv2680_q_val,Rv2680_log2FC,Rv1432_q_val,Rv1432_log2FC,Rv3594_q_val,Rv3594_log2FC,Rv3005c_q_val,Rv3005c_log2FC
0,Rv0001,dnaA,0.004064,-3.378401,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.00,1.00000,-0.10,1.00000,3.83,1.0,0.00,0.75078,3.37
1,Rv0002,dnaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.00,1.00000,0.00,1.00000,0.00,1.0,0.00,1.00000,0.00
2,Rv0003,recF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.39,0.42468,-1.93,0.69974,1.50,1.0,0.31,0.49222,3.45
3,Rv0004,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.54,1.00000,0.00,1.00000,0.00,1.0,1.79,1.00000,0.00
4,Rv0005,gyrB,0.000004,-5.919522,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.27,0.39119,-2.30,0.46795,-5.08,1.0,0.70,0.38872,-5.19


In [175]:
lfc_cols=list(rawdata.filter(regex='log2FC').columns)
qval_cols=list(rawdata.filter(regex='q_val').columns)
expt_names=[x[:-7] for x in lfc_cols]

In [176]:
#these contain log2fc and qval for few genes. Make log2FC 0 and qval 1 for the rest
setA=['2003A_Sassetti', '2003B_Sassetti', '2011_Griffin_2', '2013_Zhang_2', '2013_Zhang_1', '2013_Zhang_3A', '2013_Zhang_3B', '2013_Zhang_3C', '2013_Zhang_3D', '2018_Rittershaus_1A', '2018_Rittershaus_1B']
for exp in setA:
    rawdata[exp+'_log2FC']=rawdata[exp+'_log2FC'].fillna(0)
    rawdata[exp+'_q_val']=rawdata[exp+'_q_val'].fillna(1.0)
    rawdata[exp+'_q_val']=rawdata[exp+'_q_val'].replace('no replicates', 1.0)


In [177]:
#these contain only q values for select genes and no log2fc. Make log2FC -1 for the select and 0 for everyone else
setB=['2016_Korte', '2012_Zhang', '2015_Mendum']
for exp in setB:
    rawdata.loc[~rawdata[exp+'_q_val'].isna(),exp+'_log2FC']=-1.0
    rawdata[exp+'_log2FC']=rawdata[exp+'_log2FC'].fillna(0)
    rawdata[exp+'_q_val']=rawdata[exp+'_q_val'].fillna(1.0)
    

In [178]:
#these contain only log2fc for select and no qval. Make qval 0.05 for select and 1 for the rest
setC=['2017_Mishra_1A', '2017_Mishra_1B', '2017_Mishra_1C', '2017_Mishra_1D']
for exp in setC:
    rawdata.loc[~rawdata[exp+'_log2FC'].isna(),exp+'_q_val']=0.05
    rawdata[exp+'_log2FC']=rawdata[exp+'_log2FC'].fillna(0)
    rawdata[exp+'_q_val']=rawdata[exp+'_q_val'].fillna(1.0)


In [179]:
#these contain neither qval nor log2FC, but can use information from binary
setD=['2005_Rengarajan', '2006_Joshi_GI_1', '2006_Joshi_GI_2']
for exp in setD:
    rawdata[exp+'_log2FC']=binary[exp]
    rawdata[exp+'_log2FC']=rawdata[exp+'_log2FC'].replace(1, -1)
    rawdata.loc[rawdata[exp+'_log2FC']==-1,exp+'_q_val']=0.05
    rawdata[exp+'_q_val']=rawdata[exp+'_q_val'].fillna(1.0)


In [180]:
na_counts=rawdata.isna().sum(axis=0)
na_counts[na_counts>0]

2015_Kieser_GI_1_q_val      35
2015_Kieser_GI_1_log2FC     35
2015_Kieser_GI_2_q_val      35
2015_Kieser_GI_2_log2FC     35
2015_Kieser_GI_3_q_val      35
2015_Kieser_GI_3_log2FC     35
2016_Nambi_q_val           892
2016_Nambi_log2FC          425
2017_Xu_1A_q_val            19
2017_Xu_1A_log2FC           19
2017_Xu_1B_q_val            19
2017_Xu_1B_log2FC           19
2017_Xu_1C_q_val            19
2017_Xu_1C_log2FC           19
2017_Xu_1D_q_val            19
2017_Xu_1D_log2FC           19
2017_Xu_1E_q_val            19
2017_Xu_1E_log2FC           19
2018_Carey_1A_q_val        593
2018_Carey_1A_log2FC       593
2018_Carey_1B_q_val        582
2018_Carey_1B_log2FC       582
2018_Carey_1C_q_val        598
2018_Carey_1C_log2FC       598
2018_Carey_1D_q_val        602
2018_Carey_1D_log2FC       602
2018_Carey_1E_q_val        587
2018_Carey_1E_log2FC       587
2018_Carey_1F_q_val        593
2018_Carey_1F_log2FC       593
2018_Carey_1G_q_val        586
2018_Carey_1G_log2FC       586
2018_Car

In [181]:
#replace remaning na's and extreme vals
for col in lfc_cols:
    rawdata[col]=rawdata[col].fillna(0)
    rawdata[col]=rawdata[col].replace(-np.inf, -1)
for col in qval_cols:
    rawdata[col]=rawdata[col].fillna(1)
    rawdata[col]=rawdata[col].replace(0, 0.00001)

In [182]:
na_counts=rawdata.isna().sum(axis=0)
na_counts[na_counts>0]

Series([], dtype: int64)

In [184]:
rawdata.to_csv(pathlib.Path.cwd().parents[0].joinpath('data','Tn_library_DB_qval_log2FC_imputed.csv'), index=False)

In [31]:
lfc=rawdata[['Rv_ID']+lfc_cols]
lfc=lfc.rename(columns={x:x[:-7] for x in lfc_cols})
lfc.head()

In [32]:
qval=rawdata[['Rv_ID']+qval_cols]
qval=qval.rename(columns={x:x[:-6] for x in qval_cols})
qval.head()

In [34]:
assert all(qval.columns==lfc.columns)

True

In [42]:
lfc=lfc.dropna(axis=1, how='all')
qval=qval.dropna(axis=1, how='all')
assert all(qval.columns==lfc.columns)

ValueError: Lengths must match to compare

In [43]:
lfc.columns

Index(['Rv_ID', '2003A_Sassetti', '2003B_Sassetti', '2011_Griffin_2',
       '2013_Zhang_1', '2013_Zhang_2', '2013_Zhang_3A', '2013_Zhang_3B',
       '2013_Zhang_3C', '2013_Zhang_3D', '2017_Mishra_1A', '2017_Mishra_1B',
       '2017_Mishra_1C', '2017_Mishra_1D', '2018_Rittershaus_1A',
       '2018_Rittershaus_1B', '2015_Kieser_GI_1', '2015_Kieser_GI_2',
       '2015_Kieser_GI_3', '2016_Nambi', '2017_Xu_1A', '2017_Xu_1B',
       '2017_Xu_1C', '2017_Xu_1D', '2017_Xu_1E', '2018_Carey_1A',
       '2018_Carey_1B', '2018_Carey_1C', '2018_Carey_1D', '2018_Carey_1E',
       '2018_Carey_1F', '2018_Carey_1G', '2018_Carey_1H', '2017B_DeJesus_1A',
       '2017B_DeJesus_1B', '2017B_DeJesus_1C', 'marP', 'Rv0950', 'Rv3916c',
       'Rv0307c', 'Rv3717', 'ponA1', 'Rv1565c', 'Rv3684', 'Rv0954', 'Rv3811',
       'Rv1096', 'Rv2680', 'Rv1432', 'Rv3594', 'Rv3005c'],
      dtype='object')

In [47]:
len(qval_cols)

56

In [18]:
orf_details=orf_details[['Locus', 'Functional_Category']]

In [20]:
orf_details.head()

,Locus,Functional_Category
0,Rv3728,cell wall and cell processes
1,Rv3729,intermediary metabolism and respiration
2,Rv3730c,conserved hypotheticals
3,Rv3760,cell wall and cell processes
4,Rv3732,conserved hypotheticals


In [25]:
joined=pd.merge(lfc_raw, orf_details, left_on='Rv_ID', right_on='Locus', how='left')
joined.head()

,Rv_ID,gene_name,2012_Zhang_log2FC,2015_Kieser_GI_1_log2FC,2015_Kieser_GI_2_log2FC,2015_Kieser_GI_3_log2FC,2015_Mendum_log2FC,2016_Nambi_log2FC,2017_Xu_1A_log2FC,2017_Xu_1B_log2FC,...,Rv3684_log2FC,Rv0954_log2FC,Rv3811_log2FC,Rv1096_log2FC,Rv2680_log2FC,Rv1432_log2FC,Rv3594_log2FC,Rv3005c_log2FC,Locus,Functional_Category
0,Rv0001,dnaA,NaN,1.242756,1.584963,-0.213880,NaN,NaN,0.000000,0.000000,...,0.00,0.43,0.00,1.18,0.00,2.21,0.00,3.37,Rv0001,information pathways
1,Rv0002,dnaN,NaN,-0.574319,1.584963,-1.772497,NaN,NaN,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,2.21,0.00,0.00,Rv0002,information pathways
2,Rv0003,recF,NaN,0.913190,0.439142,-0.452153,NaN,0.599232,-0.535596,-0.549777,...,0.31,0.91,-2.59,0.48,0.03,-0.40,0.17,3.45,Rv0003,information pathways
3,Rv0004,-,NaN,0.021994,-1.119820,-2.945263,NaN,1.515639,0.000000,0.000000,...,1.79,0.00,0.00,2.61,-2.60,-2.66,1.08,0.00,Rv0004,conserved hypotheticals
4,Rv0005,gyrB,NaN,3.108722,4.087463,-0.905491,NaN,-1.948935,0.000000,0.000000,...,0.70,-5.61,-7.69,0.04,0.13,-0.97,-0.71,-5.19,Rv0005,information pathways


In [27]:
joined=joined.drop(columns=['gene_name', 'Locus'])

## Cleanup

In [33]:
joined=joined.dropna(axis=1, how='all')
joined=joined.fillna(0)
joined.head()

,Rv_ID,2015_Kieser_GI_1_log2FC,2015_Kieser_GI_2_log2FC,2015_Kieser_GI_3_log2FC,2016_Nambi_log2FC,2017_Xu_1A_log2FC,2017_Xu_1B_log2FC,2017_Xu_1C_log2FC,2017_Xu_1D_log2FC,2017_Xu_1E_log2FC,...,Rv1565c_log2FC,Rv3684_log2FC,Rv0954_log2FC,Rv3811_log2FC,Rv1096_log2FC,Rv2680_log2FC,Rv1432_log2FC,Rv3594_log2FC,Rv3005c_log2FC,Functional_Category
0,Rv0001,1.242756,1.584963,-0.213880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.43,0.00,1.18,0.00,2.21,0.00,3.37,information pathways
1,Rv0002,-0.574319,1.584963,-1.772497,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,2.21,0.00,0.00,information pathways
2,Rv0003,0.913190,0.439142,-0.452153,0.599232,-0.535596,-0.549777,0.193014,-0.210878,-0.738792,...,-1.20,0.31,0.91,-2.59,0.48,0.03,-0.40,0.17,3.45,information pathways
3,Rv0004,0.021994,-1.119820,-2.945263,1.515639,0.000000,0.000000,-0.321928,-0.152003,0.000000,...,-6.83,1.79,0.00,0.00,2.61,-2.60,-2.66,1.08,0.00,conserved hypotheticals
4,Rv0005,3.108722,4.087463,-0.905491,-1.948935,0.000000,0.000000,0.000000,0.289507,0.000000,...,-0.08,0.70,-5.61,-7.69,0.04,0.13,-0.97,-0.71,-5.19,information pathways


In [44]:
lfc_v1=joined.replace(-np.inf, -1)

In [61]:
lfc_v1['Functional_Category']=lfc_v1['Functional_Category'].replace(0, 'unknown')

## EDA

In [42]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from plotnine import *

In [95]:
lfc=lfc_v1.copy(deep=True)

In [63]:
lfc_x=lfc.drop(columns=['Rv_ID', 'Functional_Category']).values
lfc_y=lfc['Functional_Category'].values

In [64]:
lfc_x=StandardScaler().fit_transform(lfc_x)

In [65]:
pca=PCA(n_components=3)
principalComponents=pca.fit_transform(lfc_x)

In [66]:
pca.explained_variance_ratio_

array([0.14126027, 0.10559412, 0.08442941])

## Log reg

In [67]:
from sklearn.linear_model import LogisticRegression

In [100]:
lfc=lfc[~lfc['Functional_Category'].isin(['conserved hypotheticals', 'unknown'])]
lfc_x=lfc.drop(columns=['Rv_ID', 'Functional_Category']).values
lfc_y=lfc['Functional_Category'].values

In [117]:
clf=LogisticRegression(penalty='l1')
clf.fit(lfc_x,lfc_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [115]:
a=clf.predict_proba(lfc_x)

In [116]:
clf.score(lfc_x, lfc_y)

0.3256756756756757

In [87]:
lfc['Functional_Category'].value_counts()

conserved hypotheticals                    1025
intermediary metabolism and respiration     937
cell wall and cell processes                771
lipid metabolism                            273
information pathways                        244
virulence, detoxification, adaptation       222
regulatory proteins                         201
PE/PPE                                      166
insertion seqs and phages                   146
unknown                                      17
Name: Functional_Category, dtype: int64

In [122]:
a=clf.coef_

array([ 0.00528255, -0.00391622,  0.02277458,  0.00049432,  0.04531478,
        0.13041979, -0.0367974 , -0.00355048,  0.01569048,  0.00946108,
        0.04692735,  0.02425045, -0.0483615 , -0.02338595,  0.00108854,
       -0.0568301 , -0.0684027 ,  0.02174038,  0.00197637, -0.01877926,
        0.00575424, -0.00861422, -0.04360628,  0.01980476, -0.00294847,
        0.00896251,  0.00959415, -0.01125965,  0.00140207, -0.00046203,
       -0.00043935,  0.1404315 , -0.00682654, -0.02280513, -0.00369418])